In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
ERA5_ds = xr.open_dataset('/work/mh0033/m301036/LSAT/Data/ERA5/ERA20C-ERA5_190001-202212_rg.nc')
ERA5 =ERA5_ds.sel(time=slice('1958-01-01','2022-12-31'),lat=slice(0,90))
ERA5 = ERA5.assign_attrs(units='hPa') # fixed line
ERA5 = ERA5/100.0
ERA5

<xarray.Dataset>
Dimensions:  (time: 780, lon: 180, lat: 45)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01 1958-02-01 ... 2022-12-01
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Data variables:
    msl      (time, lat, lon) float32 1.011e+03 1.011e+03 ... 1.022e+03
Attributes:
    CDI:          Climate Data Interface version 2.0.6 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Jul 26 13:00:50 2023: cdo remapbil,r180x90 /work/mh0033...
    CDO:          Climate Data Operators version 2.0.6 (https://mpimet.mpg.de...
    units:        hPa

In [3]:
#Covert the 0-360 to -180-180
ERA5.coords['lon'] = (ERA5.coords['lon'] + 180) % 360 - 180
ERA5 = ERA5.sortby(ERA5.lon)
ERA5

<xarray.Dataset>
Dimensions:  (time: 780, lon: 180, lat: 45)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01 1958-02-01 ... 2022-12-01
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Data variables:
    msl      (time, lat, lon) float32 1.009e+03 1.009e+03 ... 1.023e+03
Attributes:
    CDI:          Climate Data Interface version 2.0.6 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Jul 26 13:00:50 2023: cdo remapbil,r180x90 /work/mh0033...
    CDO:          Climate Data Operators version 2.0.6 (https://mpimet.mpg.de...
    units:        hPa

In [4]:
ERA5_climatology = ERA5.groupby('time.month').mean(dim='time')
ERA5_climatology
ERA5_ano = ERA5.groupby('time.month') - ERA5_climatology
ERA5_ano

<xarray.Dataset>
Dimensions:  (time: 780, lon: 180, lat: 45)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01 1958-02-01 ... 2022-12-01
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    msl      (time, lat, lon) float32 0.8057 0.6676 0.5375 ... 8.451 8.475 8.498

In [5]:
ERA5_ano['DJF'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='DJF').groupby('time.year').mean('time')
ERA5_ano['MAM'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='MAM').groupby('time.year').mean('time')
ERA5_ano['JJA'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='JJA').groupby('time.year').mean('time')
ERA5_ano['SON'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='SON').groupby('time.year').mean('time')

## Define function

In [6]:
from scipy.stats import linregress

def calc_trend(data):
    slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(data)), data)  
    return slope, p_value

## ERA5 seasonal trend pattern

In [7]:
ERA5_ano['DJF_trend'], ERA5_ano['p_value_DJF'] = xr.apply_ufunc(calc_trend, ERA5_ano['DJF'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['MAM_trend'], ERA5_ano['p_value_MAM'] = xr.apply_ufunc(calc_trend, ERA5_ano['MAM'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['JJA_trend'], ERA5_ano['p_value_JJA'] = xr.apply_ufunc(calc_trend, ERA5_ano['JJA'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['SON_trend'], ERA5_ano['p_value_SON'] = xr.apply_ufunc(calc_trend, ERA5_ano['SON'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)

In [8]:
SLP_trend = xr.Dataset({
    'DJF': ERA5_ano['DJF_trend']*65.0,
    'MAM': ERA5_ano['MAM_trend']*65.0,
    'JJA': ERA5_ano['JJA_trend']*65.0,
    'SON': ERA5_ano['SON_trend']*65.0
}, coords={'lon': ERA5_ano['lon'], 'lat': ERA5_ano['lat'], 'season': ['DJF', 'MAM', 'JJA', 'SON']})
SLP_trend

<xarray.Dataset>
Dimensions:  (lon: 180, lat: 45, season: 4)
Coordinates:
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
  * season   (season) <U3 'DJF' 'MAM' 'JJA' 'SON'
Data variables:
    DJF      (lat, lon) float64 0.3514 0.3846 0.4154 ... -1.051 -1.063 -1.075
    MAM      (lat, lon) float64 0.6791 0.6921 0.7049 ... -2.483 -2.472 -2.46
    JJA      (lat, lon) float64 0.5895 0.5984 0.6004 ... -1.402 -1.392 -1.381
    SON      (lat, lon) float64 0.5828 0.5926 0.594 ... -1.437 -1.434 -1.429

In [9]:
sig_ds = xr.Dataset({
    'DJF': ERA5_ano['p_value_DJF'],
    'MAM': ERA5_ano['p_value_MAM'],
    'JJA': ERA5_ano['p_value_JJA'],
    'SON': ERA5_ano['p_value_SON']
}, coords={'lon': ERA5_ano['lon'], 'lat': ERA5_ano['lat'], 'season': ['DJF', 'MAM', 'JJA', 'SON']})
sig_ds

<xarray.Dataset>
Dimensions:  (lon: 180, lat: 45, season: 4)
Coordinates:
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
  * season   (season) <U3 'DJF' 'MAM' 'JJA' 'SON'
Data variables:
    DJF      (lat, lon) float64 0.1055 0.07757 0.05935 ... 0.5906 0.586 0.5815
    MAM      (lat, lon) float64 0.0005891 0.0003859 0.0002675 ... 0.1958 0.1974
    JJA      (lat, lon) float64 0.002542 0.002628 0.003179 ... 0.3799 0.3838
    SON      (lat, lon) float64 0.001553 0.001508 0.001779 ... 0.324 0.3252

## MPI-ESM-Trend Calculating

In [10]:
data_hist_ssp245_MPI_ESM  = '/work/mh0033/m301036/LSAT/CMIP6-MPI-M-LR/MergeDataOut/psl_Amon_1850-2022_*.nc'
ds = xr.open_mfdataset(data_hist_ssp245_MPI_ESM, combine = 'nested', concat_dim = 'run')
ds

<xarray.Dataset>
Dimensions:    (time: 2076, run: 30, bnds: 2, lon: 180, lat: 90)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon        (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
Dimensions without coordinates: run, bnds
Data variables:
    time_bnds  (run, time, bnds) datetime64[ns] dask.array<chunksize=(1, 2076, 2), meta=np.ndarray>
    psl        (run, time, lat, lon) float32 dask.array<chunksize=(1, 2076, 90, 180), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.3 (https://mpi...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            psl
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/2d4e90e1-d70a-49f4-8374-eb769a0b143d
    CDO:                    Climate Data Operators version 2.0.3 (https://mpi...

In [11]:
# Convert from 0-360 to -180-180
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)
ds

<xarray.Dataset>
Dimensions:    (time: 2076, run: 30, bnds: 2, lon: 180, lat: 90)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon        (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
Dimensions without coordinates: run, bnds
Data variables:
    time_bnds  (run, time, bnds) datetime64[ns] dask.array<chunksize=(1, 2076, 2), meta=np.ndarray>
    psl        (run, time, lat, lon) float32 dask.array<chunksize=(1, 2076, 90, 180), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.3 (https://mpi...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            psl
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/2d4e90e1-d70a-49f4-8374-eb769a0b143d
    CDO:                    Climate Data Operators version 2.0.3 (https://mpi...

In [12]:
ds['psl'] = ds['psl']/100.0
ds['psl'] = ds['psl'].assign_attrs(units='hPa')
ds

<xarray.Dataset>
Dimensions:    (time: 2076, run: 30, bnds: 2, lon: 180, lat: 90)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon        (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
Dimensions without coordinates: run, bnds
Data variables:
    time_bnds  (run, time, bnds) datetime64[ns] dask.array<chunksize=(1, 2076, 2), meta=np.ndarray>
    psl        (run, time, lat, lon) float32 dask.array<chunksize=(1, 2076, 90, 180), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.3 (https://mpi...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            psl
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/2d4e90e1-d70a-49f4-8374-eb769a0b143d
    CDO:                    Climate Data Operators version 2.0.3 (https://mpi...

In [13]:
slp = ds['psl'].loc[:,'1958-01-01':'2022-12-31',0:90,:]
print(slp.min().values)
slp_climatology = slp.groupby('time.month').mean(dim='time')
slp_climatology
slp_ano = slp.groupby('time.month') - slp_climatology
slp_ano_ds = slp_ano.to_dataset()
slp_ano_ds

972.03156


/home/m/m301036/.conda/envs/mykernel/lib/python3.9/site-packages/xarray/core/indexing.py:1440: PerformanceWarning: Slicing with an out-of-order index is generating 65 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:  (time: 780, lon: 180, lat: 45, run: 30)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: run
Data variables:
    psl      (run, time, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>

In [14]:
seasons = ['JJA', 'DJF', 'MAM', 'SON']
season_means = {}

for season in seasons:
    if season == 'JJA':
        months = [6,7,8]
    elif season == 'DJF':
        months =[12,1,2]
    elif season == 'MAM':
        months = [3,4,5]
    elif season == 'SON':
        months = [9,10,11]

    season_months = slp_ano_ds.sel(time=slp_ano_ds.time.dt.month.isin(months))
    
    # Calculate the seasonal mean SAT anomalies
    season_mean_anomalies = season_months.groupby('time.year').mean('time')
    
    # Store the seasonal mean in the dictionary
    season_means[season] = season_mean_anomalies['psl']

# Access the multiyear JJA mean SAT anomalies
    
slp_ano_ds['JJA'] = season_means['JJA']
slp_ano_ds['DJF'] = season_means['DJF']
slp_ano_ds['MAM'] = season_means['MAM']
slp_ano_ds['SON'] = season_means['SON']

slp_ano_ds

<xarray.Dataset>
Dimensions:  (time: 780, lon: 180, lat: 45, run: 30, year: 65)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
  * year     (year) int64 1958 1959 1960 1961 1962 ... 2018 2019 2020 2021 2022
Dimensions without coordinates: run
Data variables:
    psl      (run, time, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    JJA      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    DJF      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    MAM      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    SON      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>

In [15]:
import scipy.stats as stats
from scipy.stats import linregress

def calc_trend(data):
    slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(data)), data)  
    return slope, p_value

In [16]:
slp_ano_ds['slope_JJA'], slp_ano_ds['p_value_JJA'] = xr.apply_ufunc(calc_trend, slp_ano_ds['JJA'].chunk(dict(run=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_JJA'].attrs['units'] = 'Pa/65yrs'
slp_ano_ds['p_value_JJA'].attrs['units'] = 'p_value'

In [17]:
slp_ano_ds['slope_DJF'], slp_ano_ds['p_value_DJF'] = xr.apply_ufunc(calc_trend, slp_ano_ds['DJF'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_DJF'].attrs['units'] = 'Pa/65yrs'
slp_ano_ds['p_value_DJF'].attrs['units'] = 'p_value'

In [18]:
slp_ano_ds['slope_MAM'], slp_ano_ds['p_value_MAM'] = xr.apply_ufunc(calc_trend, slp_ano_ds['MAM'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_MAM'].attrs['units'] = 'Pa/65yrs'
slp_ano_ds['p_value_MAM'].attrs['units'] = 'p_value'

In [19]:
slp_ano_ds['slope_SON'], slp_ano_ds['p_value_SON'] = xr.apply_ufunc(calc_trend, slp_ano_ds['SON'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_SON'].attrs['units'] = 'Pa/65yrs'
slp_ano_ds['p_value_SON'].attrs['units'] = 'p_value'

In [20]:
slp_ano_ds = slp_ano_ds.compute()

In [21]:
slp_ano_ds

<xarray.Dataset>
Dimensions:      (time: 780, lon: 180, lat: 45, run: 30, year: 65)
Coordinates:
  * time         (time) datetime64[ns] 1958-01-16T12:00:00 ... 2022-12-16T12:...
  * lon          (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
  * lat          (lat) float64 1.0 3.0 5.0 7.0 9.0 ... 81.0 83.0 85.0 87.0 89.0
    month        (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 3 4 5 6 7 8 9 10 11 12
  * year         (year) int64 1958 1959 1960 1961 1962 ... 2019 2020 2021 2022
Dimensions without coordinates: run
Data variables: (12/13)
    psl          (run, time, lat, lon) float32 0.9214 0.9156 ... -4.258 -4.234
    JJA          (year, run, lat, lon) float32 0.06419 0.084 ... -2.481 -2.482
    DJF          (year, run, lat, lon) float32 0.9636 0.9672 ... 7.061 7.069
    MAM          (year, run, lat, lon) float32 0.2683 0.2927 ... -5.174 -5.158
    SON          (year, run, lat, lon) float32 -0.4274 -0.4326 ... 2.781 2.761
    slope_JJA    (run, lat, lon) float64 -0.002578 -0.002453 ... -0.0006864
    ...           ...
    slope_DJF    (run, lat, lon) float64 -0.005107 -0.00501 ... -0.03655
    p_value_DJF  (run, lat, lon) float64 0.1101 0.123 0.1572 ... 0.2493 0.2462
    slope_MAM    (run, lat, lon) float64 -0.001661 -0.001676 ... 0.04081 0.0408
    p_value_MAM  (run, lat, lon) float64 0.5429 0.5413 ... 0.09561 0.09513
    slope_SON    (run, lat, lon) float64 0.004975 0.004881 ... 0.007394 0.007392
    p_value_SON  (run, lat, lon) float64 0.1272 0.1396 0.1309 ... 0.697 0.6968

## Calculate the SLP pattern corrlation

In [22]:
#define thr function to calculate the pattern correlation
from scipy.stats import pearsonr
def pattern_correlation(x, y):
    """ Pattern Correlation function """
    # Ensure that data are properly alinged to each other
    #convert the two dimensions to the one dimension without the nan values
    
    x_flat = x.flatten()
    y_flat = y.flatten()

    #remove the nan values
    x_flat = x_flat[~np.isnan(x_flat)]
    y_flat = y_flat[~np.isnan(y_flat)]
    # x,y = xr.align(x,y)
    corr, pval = pearsonr(x_flat,y_flat)

    return corr, pval

In [23]:
# Define the output core dimensions for the pattern_correlation function
output_core_dims = [[], []]
input_core_dims = [['lat', 'lon'], ['lat', 'lon']]
# Calculate the pattern correlation for DJF
pattern_corr_DJF, pval_DJF = xr.apply_ufunc(pattern_correlation, SLP_trend['DJF'], slp_ano_ds['slope_DJF'], input_core_dims=input_core_dims, output_core_dims=output_core_dims, vectorize=True, dask='parallelized')
pattern_corr_DJF.attrs['units'] = 'pattern correlation'
pattern_corr_DJF

<xarray.DataArray (run: 30)>
array([-0.37682027, -0.5349629 ,  0.39176266,  0.30227844,  0.18713311,
        0.30855548,  0.59974265, -0.46397319, -0.29876546,  0.3564727 ,
       -0.27792427, -0.47733327,  0.17541802,  0.06529977, -0.02030236,
        0.49219613, -0.28651452, -0.41161646,  0.34466418, -0.40157779,
       -0.04269701,  0.45385107,  0.3004036 , -0.160038  ,  0.5879287 ,
        0.05011435,  0.35037329,  0.19072075, -0.23155474,  0.36910794])
Dimensions without coordinates: run
Attributes:
    units:    pattern correlation

In [24]:
pattern_corr_MAM, pval_MAM = xr.apply_ufunc(pattern_correlation, SLP_trend['MAM'], slp_ano_ds['slope_MAM'], input_core_dims=input_core_dims, output_core_dims=output_core_dims, vectorize=True)
pattern_corr_MAM.attrs['units'] = 'pattern correlation'
pattern_corr_MAM

pattern_corr_JJA, pval_JJA = xr.apply_ufunc(pattern_correlation, SLP_trend['JJA'], slp_ano_ds['slope_JJA'], input_core_dims=input_core_dims, output_core_dims=output_core_dims, vectorize=True)
pattern_corr_JJA.attrs['units'] = 'pattern correlation'
pattern_corr_JJA

pattern_corr_SON, pval_SON = xr.apply_ufunc(pattern_correlation, SLP_trend['SON'], slp_ano_ds['slope_SON'], input_core_dims=input_core_dims, output_core_dims=output_core_dims, vectorize=True)
pattern_corr_SON.attrs['units'] = 'pattern correlation'
pattern_corr_SON

<xarray.DataArray (run: 30)>
array([ 0.44393784,  0.39675211,  0.34553718,  0.44794587,  0.36943404,
        0.47899911,  0.12693581,  0.66087086,  0.40397828,  0.4376726 ,
        0.56251803,  0.05211482,  0.19576596,  0.39742963,  0.55690858,
        0.26300472,  0.18731795,  0.47787394,  0.58985552,  0.42499196,
        0.68063916,  0.06895965,  0.24686263,  0.02848133,  0.46400091,
        0.25623134, -0.10854848,  0.34308075,  0.68413811, -0.27433156])
Dimensions without coordinates: run
Attributes:
    units:    pattern correlation

In [25]:
pattern_corr_DJF = pattern_corr_DJF.compute()
pattern_corr_MAM = pattern_corr_MAM.compute()
pattern_corr_JJA = pattern_corr_JJA.compute()
pattern_corr_SON = pattern_corr_SON.compute()

In [26]:
pattern_corr_DJF

<xarray.DataArray (run: 30)>
array([-0.37682027, -0.5349629 ,  0.39176266,  0.30227844,  0.18713311,
        0.30855548,  0.59974265, -0.46397319, -0.29876546,  0.3564727 ,
       -0.27792427, -0.47733327,  0.17541802,  0.06529977, -0.02030236,
        0.49219613, -0.28651452, -0.41161646,  0.34466418, -0.40157779,
       -0.04269701,  0.45385107,  0.3004036 , -0.160038  ,  0.5879287 ,
        0.05011435,  0.35037329,  0.19072075, -0.23155474,  0.36910794])
Dimensions without coordinates: run
Attributes:
    units:    pattern correlation

In [27]:
# Save the pattern correlation data array to a netCDF file
pattern_corr_DJF.to_netcdf('/work/mh0033/m301036/josie/LSAT/1850-2100Analyses/SpatialPattern/patternCorrelation/SLP_corr_DJF_MPI-ESM.nc')
pattern_corr_MAM.to_netcdf('/work/mh0033/m301036/josie/LSAT/1850-2100Analyses/SpatialPattern/patternCorrelation/SLP_corr_MAM_MPI-ESM.nc')
pattern_corr_JJA.to_netcdf('/work/mh0033/m301036/josie/LSAT/1850-2100Analyses/SpatialPattern/patternCorrelation/SLP_corr_JJA_MPI-ESM.nc')
pattern_corr_SON.to_netcdf('/work/mh0033/m301036/josie/LSAT/1850-2100Analyses/SpatialPattern/patternCorrelation/SLP_corr_SON_MPI-ESM.nc')

## Selection of the minimum and maximum five models

In [ ]:
model_65yr_JJA = np.genfromtxt(fname='/home/m/m301036/josie/LSAT/1850-2100Analyses/MPI-ESM-LR_NH_SATAs_65yr_JJA_trend.txt',delimiter='\t', skip_header=1)
model_65yr_DJF = np.genfromtxt(fname='/home/m/m301036/josie/LSAT/1850-2100Analyses/MPI-ESM-LR_NH_SATAs_65yr_DJF_trend.txt',delimiter='\t', skip_header=1)

DJF_65yr = xr.DataArray(model_65yr_DJF[:,1], dims=['run'], coords={'run': np.arange(1, 31, 1)})
JJA_65yr = xr.DataArray(model_65yr_JJA[:,1], dims=['run'], coords={'run': np.arange(1, 31, 1)})

# assume you have an xarray called 'xr_data'
# get the most minimum five values and their indices
min_indices = np.argpartition(DJF_65yr.values.flatten(), 5)[:5]
min_values = DJF_65yr.values.flatten()[min_indices]
min_coords = np.unravel_index(min_indices, DJF_65yr.shape)

# create a new xarray to store the most minimum values
min_xr = xr.DataArray(min_values, dims=['value'], coords={'value': np.arange(5)})

# print the new xarray
print(min_xr)
print(min_coords)

<xarray.DataArray (value: 5)>
array([1.36645001, 1.40558001, 1.41727347, 1.42440782, 1.4636625 ])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([19, 12,  9, 26, 16]),)


In [ ]:
max_indices = np.argpartition(DJF_65yr.values.flatten(), -5)[-5:]
max_values = DJF_65yr.values.flatten()[max_indices]
max_coords = np.unravel_index(max_indices, DJF_65yr.shape)

max_xr = xr.DataArray(max_values, dims=['value'], coords={'value': np.arange(5)})
print(max_xr)
print(max_coords)
type(max_coords)
max_coords

<xarray.DataArray (value: 5)>
array([1.89248708, 2.09459221, 2.24413737, 1.93526643, 1.98230888])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([23, 24, 27, 11, 17]),)


(array([23, 24, 27, 11, 17]),)

In [ ]:
max_JJA_indices = np.argpartition(JJA_65yr.values.flatten(), -5)[-5:]
max_JJA_values = JJA_65yr.values.flatten()[max_JJA_indices]
max_JJA_coords = np.unravel_index(max_JJA_indices, JJA_65yr.shape)

max_JJA_xr = xr.DataArray(max_JJA_values, dims=['value'], coords={'value': np.arange(5)})
print(max_JJA_xr)
print(max_JJA_coords)
type(max_JJA_coords)
max_JJA_coords

<xarray.DataArray (value: 5)>
array([1.62323888, 1.63127335, 1.65849766, 1.66249305, 1.71862692])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([16, 10, 27, 19, 23]),)


(array([16, 10, 27, 19, 23]),)

In [ ]:
min_JJA_indices = np.argpartition(JJA_65yr.values.flatten(), 5)[:5]
min_JJA_values = JJA_65yr.values.flatten()[min_JJA_indices]
min_JJA_coords = np.unravel_index(min_JJA_indices, JJA_65yr.shape)

min_JJA_xr = xr.DataArray(min_JJA_values, dims=['value'], coords={'value': np.arange(5)})
print(min_JJA_xr)
print(min_JJA_coords)
type(min_JJA_coords)
min_JJA_coords

<xarray.DataArray (value: 5)>
array([1.39847864, 1.41214496, 1.3810671 , 1.32235258, 1.43317934])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([ 4, 20,  8,  7, 13]),)


(array([ 4, 20,  8,  7, 13]),)

## Plot the trend pattern of the DJF 65-yr Spatial pattern

In [ ]:
slope_JJA = slp_ano_ds['slope_JJA'] 
slope_DJF = slp_ano_ds['slope_DJF']
slope_MAM = slp_ano_ds['slope_MAM']
slope_SON = slp_ano_ds['slope_SON']

p_value_JJA = slp_ano_ds['p_value_JJA']
p_value_DJF = slp_ano_ds['p_value_DJF']
p_value_MAM = slp_ano_ds['p_value_MAM']
p_value_SON = slp_ano_ds['p_value_SON']

In [ ]:
slope_JJA_data = slope_JJA*65.0
slope_DJF_data = slope_DJF*65.0
slope_MAM_data = slope_MAM*65.0
slope_SON_data = slope_SON*65.0

In [ ]:
slope_DJF_data

<xarray.DataArray 'slope_DJF' (run: 30, lat: 45, lon: 180)>
array([[[-1.53227018e+01, -1.64314037e+01, -1.52088365e+01, ...,
         -1.16662821e+01, -1.41563685e+01, -1.58477455e+01],
        [-1.15127022e+01, -1.54840349e+01, -1.42825087e+01, ...,
         -9.07438129e+00, -1.29887178e+01, -1.20908283e+01],
        [ 2.99127785e+00, -5.20136388e+00, -5.65773731e+00, ...,
          1.70077153e+00, -1.28092437e+00,  2.99313455e+00],
        ...,
        [ 2.76604192e+02,  2.78021628e+02,  2.79197939e+02, ...,
          2.70973429e+02,  2.73087000e+02,  2.74974613e+02],
        [ 2.86938363e+02,  2.88666360e+02,  2.90292821e+02, ...,
          2.81133534e+02,  2.83141322e+02,  2.85086017e+02],
        [ 2.81606281e+02,  2.82297606e+02,  2.82968885e+02, ...,
          2.78692116e+02,  2.79438282e+02,  2.80176533e+02]],

       [[ 2.59449277e+01,  2.49596797e+01,  2.47383993e+01, ...,
          2.69872081e+01,  2.60480960e+01,  2.46816106e+01],
        [ 2.97102573e+01,  2.53464134e+01,  2.55932616e+01, ...,
          2.88875767e+01,  2.59732338e+01,  2.80988548e+01],
        [ 4.06383972e+01,  3.47243284e+01,  3.75247769e+01, ...,
          3.80218325e+01,  3.55258862e+01,  4.03334888e+01],
...
        [ 1.94733526e+02,  1.98692407e+02,  2.02539849e+02, ...,
          1.82369362e+02,  1.86572196e+02,  1.90715750e+02],
        [ 2.54339362e+02,  2.57920405e+02,  2.61417490e+02, ...,
          2.43218367e+02,  2.46979596e+02,  2.50704874e+02],
        [ 2.85058585e+02,  2.86682754e+02,  2.88307938e+02, ...,
          2.78655257e+02,  2.80240471e+02,  2.81845847e+02]],

       [[ 2.77533364e+01,  2.72228711e+01,  2.79901889e+01, ...,
          3.22680067e+01,  2.88025042e+01,  2.65866852e+01],
        [ 3.39527401e+01,  2.96035822e+01,  3.11876114e+01, ...,
          3.80518030e+01,  3.38982524e+01,  3.44946658e+01],
        [ 5.70921890e+01,  4.58438019e+01,  4.36873900e+01, ...,
          5.16676283e+01,  5.08688233e+01,  5.84596049e+01],
        ...,
        [-1.74505328e+02, -1.76657718e+02, -1.79083677e+02, ...,
         -1.69900604e+02, -1.71100663e+02, -1.72636165e+02],
        [-2.25451216e+02, -2.25154823e+02, -2.24941716e+02, ...,
         -2.26842746e+02, -2.26279526e+02, -2.25826083e+02],
        [-2.57860013e+02, -2.56988348e+02, -2.56118264e+02, ...,
         -2.61310858e+02, -2.60449858e+02, -2.59587986e+02]]])
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Dimensions without coordinates: run

In [ ]:
slope_JJA_MME = slope_JJA_data.mean(dim='run')
slope_DJF_MME = slope_DJF_data.mean(dim='run')
slope_MAM_MME = slope_MAM_data.mean(dim='run')
slope_SON_MME = slope_SON_data.mean(dim='run')

In [ ]:
#using the for loop to pick up the min5 and max5 data of DJF 


# Extract trend spatial data for minimum five runs in DJF
DJF_min5_trend = []
for i in range(5):
    run_index = min_coords[0][i]
    DJF_min5_trend.append(slope_DJF_data[run_index,:,:])
DJF_min5_trend = xr.concat(DJF_min5_trend, dim='run')

# Extract trend spatial data for maximum five runs in DJF
DJF_max5_trend = []
for i in range(5):
    run_index = max_coords[0][i]
    DJF_max5_trend.append(slope_DJF_data[run_index,:,:])
DJF_max5_trend = xr.concat(DJF_max5_trend, dim='run')

# Extract trend spatial data for minimum five runs in JJA
JJA_min5_trend = []
for i in range(5):
    run_index = min_JJA_coords[0][i]
    JJA_min5_trend.append(slope_JJA_data[run_index,:,:])
JJA_min5_trend = xr.concat(JJA_min5_trend, dim='run')

# Extract trend spatial data for maximum five runs in JJA
JJA_max5_trend = []
for i in range(5):
    run_index = max_JJA_coords[0][i]
    JJA_max5_trend.append(slope_JJA_data[run_index,:,:])
JJA_max5_trend = xr.concat(JJA_max5_trend, dim='run')


In [ ]:
JJA_max5_trend

<xarray.DataArray 'slope_JJA' (run: 5, lat: 45, lon: 180)>
array([[[-4.88412207e+01, -4.97074902e+01, -4.73807273e+01, ...,
         -4.99864092e+01, -5.03583456e+01, -4.98532414e+01],
        [-4.74434403e+01, -4.66698697e+01, -4.47965571e+01, ...,
         -4.89724048e+01, -4.98079640e+01, -4.92412927e+01],
        [-4.54847382e+01, -4.31827058e+01, -4.24213357e+01, ...,
         -4.99508466e+01, -4.90792573e+01, -4.80131098e+01],
        ...,
        [-1.53382626e+02, -1.57911985e+02, -1.62626673e+02, ...,
         -1.41268374e+02, -1.45068299e+02, -1.49103618e+02],
        [-1.76470920e+02, -1.78834118e+02, -1.81266263e+02, ...,
         -1.69913186e+02, -1.72004716e+02, -1.74189784e+02],
        [-1.94403741e+02, -1.95145001e+02, -1.95906644e+02, ...,
         -1.91658533e+02, -1.92318634e+02, -1.92994995e+02]],

       [[-4.03442007e+01, -4.01486670e+01, -3.81234461e+01, ...,
         -4.08924299e+01, -4.16916656e+01, -4.14486345e+01],
        [-3.92723138e+01, -3.83646879e+01, -3.75395802e+01, ...,
         -4.15664367e+01, -4.27656326e+01, -4.19599167e+01],
        [-3.64620847e+01, -3.56431776e+01, -3.52615113e+01, ...,
         -4.18508452e+01, -4.15012058e+01, -3.94955381e+01],
...
        [ 1.42764335e+01,  1.22649574e+01,  1.00947196e+01, ...,
          1.91693546e+01,  1.77391767e+01,  1.61071033e+01],
        [ 8.99293692e+00,  7.86381243e+00,  6.67801675e+00, ...,
          1.20254933e+01,  1.10818174e+01,  1.00558768e+01],
        [ 9.74388547e-01,  5.51912271e-01,  1.24573908e-01, ...,
          2.51658028e+00,  2.15067694e+00,  1.77211626e+00]],

       [[-1.66895120e+01, -1.71814927e+01, -1.61272197e+01, ...,
         -1.72343237e+01, -1.86879661e+01, -1.84482279e+01],
        [-1.52002470e+01, -1.47094133e+01, -1.32588859e+01, ...,
         -1.86002534e+01, -1.88341549e+01, -1.71863756e+01],
        [-1.57183650e+01, -1.40395873e+01, -1.19329870e+01, ...,
         -2.02734812e+01, -1.91486296e+01, -1.73116559e+01],
        ...,
        [-2.60677365e+01, -2.25956353e+01, -1.91845063e+01, ...,
         -3.68766785e+01, -3.32253857e+01, -2.96083114e+01],
        [-1.20020733e+01, -9.71893221e+00, -7.45547950e+00, ...,
         -1.88887480e+01, -1.65977282e+01, -1.42801420e+01],
        [-4.30036643e+00, -3.30768365e+00, -2.32312731e+00, ...,
         -8.31604912e+00, -7.30414569e+00, -6.29299631e+00]]])
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Dimensions without coordinates: run

In [ ]:
# Calculate the DJF min 5 trend mean
DJF_min5_trend_mean = DJF_min5_trend.mean(dim='run')
DJF_max5_trend_mean = DJF_max5_trend.mean(dim='run')

# Calculate the JJA min 5 trend mean
JJA_min5_trend_mean = JJA_min5_trend.mean(dim='run')
JJA_max5_trend_mean = JJA_max5_trend.mean(dim='run')

In [37]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

# plt.style.use('ggplot')
# levels = np.arange(-2.0, 5.5, 0.5)
# sns.color_palette("rocket_r", as_cmap=True)
# cmap = plt.cm.get_cmap('rocket_r', len(levels) - 1)
# bounds = [-2, -1, 0, 1, 2, 3, 4, 5]
# norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
levels = np.arange(-700.0, 700.0, 50.0)
cmap = ListedColormap(sns.color_palette("RdBu_r", n_colors=len(levels)-1).as_hex())

fig = plt.figure(figsize=(15, 8), dpi=300)
gs = gridspec.GridSpec(nrows=2, ncols=2, height_ratios=[1, 1], width_ratios=[1, 1])

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='lightgrey')
ax1.add_feature(cfeature.OCEAN, facecolor='white')

ax1.set_title('a', loc='left', fontsize=12, fontweight='medium')
ax1.set_title('ERA5 DJF', loc='right', fontsize=12, fontweight='medium')

ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax1.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax1.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax1.contourf(SLP_trend['longitude'], SLP_trend['latitude'], SLP_trend['DJF'], levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('Pa/35yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax2.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax2.coastlines()
ax2.add_feature(cfeature.LAND, facecolor='lightgrey')
ax2.add_feature(cfeature.OCEAN, facecolor='white')

ax2.set_title('b', loc='left', fontsize=12, fontweight='medium')
ax2.set_title('MPI-ESM-LR DJF MME', loc='right', fontsize=12, fontweight='medium')

ax2.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax2.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax2.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax2.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax2.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], slope_DJF_MME, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax3.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax3.coastlines()
ax3.add_feature(cfeature.LAND, facecolor='lightgrey')
ax3.add_feature(cfeature.OCEAN, facecolor='white')

ax3.set_title('c', loc='left', fontsize=12, fontweight='medium')
ax3.set_title('MPI-ESM-LR DJF Min5-MME', loc='right', fontsize=12, fontweight='medium')

ax3.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax3.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax3.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax3.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax3.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax3.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], DJF_min5_trend_mean , levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax4.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax4.coastlines()
ax4.add_feature(cfeature.LAND, facecolor='lightgrey')
ax4.add_feature(cfeature.OCEAN, facecolor='white')

ax4.set_title('d', loc='left', fontsize=12, fontweight='medium')
ax4.set_title('MPI-ESM-LR DJF Max5-MME', loc='right', fontsize=12, fontweight='medium')

ax4.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax4.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax4.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax4.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax4.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax4.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], DJF_max5_trend_mean, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

plt.subplots_adjust(wspace=0.1, hspace=0.01, top=0.75, bottom=0.05)

cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
cb.set_label('Pa/65yrs', fontsize=10, fontweight='medium')
cb.ax.tick_params(labelsize=10)

plt.savefig('Obs-MPI-ESM-LR_1958-2022_DJF_SLP_trend_MME.png', bbox_inches='tight', dpi=300)

plt.close()

/tmp/ipykernel_284438/3576098113.py:107: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')


In [34]:
# # Define the levels and colormap
levels = np.arange(-300.0, 325.0, 25.0)
cmap = ListedColormap(sns.color_palette("RdBu_r", n_colors=len(levels)-1).as_hex())

fig = plt.figure(figsize=(15, 8), dpi=300)
gs = gridspec.GridSpec(nrows=2, ncols=2, height_ratios=[1, 1], width_ratios=[1, 1])

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='lightgrey')
ax1.add_feature(cfeature.OCEAN, facecolor='white')

ax1.set_title('a', loc='left', fontsize=12, fontweight='medium')
ax1.set_title('ERA5 JJA', loc='right', fontsize=12, fontweight='medium')

ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax1.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax1.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax1.contourf(SLP_trend['longitude'], SLP_trend['latitude'], SLP_trend['JJA'], levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax2.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax2.coastlines()
ax2.add_feature(cfeature.LAND, facecolor='lightgrey')
ax2.add_feature(cfeature.OCEAN, facecolor='white')

ax2.set_title('b', loc='left', fontsize=12, fontweight='medium')
ax2.set_title('MPI-ESM-LR JJA MME', loc='right', fontsize=12, fontweight='medium')

ax2.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax2.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax2.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax2.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax2.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], slope_JJA_MME, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax3.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax3.coastlines()
ax3.add_feature(cfeature.LAND, facecolor='lightgrey')
ax3.add_feature(cfeature.OCEAN, facecolor='white')

ax3.set_title('c', loc='left', fontsize=12, fontweight='medium')
ax3.set_title('MPI-ESM-LR JJA Min5-MME', loc='right', fontsize=12, fontweight='medium')

ax3.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax3.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax3.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax3.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax3.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax3.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], JJA_min5_trend_mean , levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax4.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax4.coastlines()
ax4.add_feature(cfeature.LAND, facecolor='lightgrey')
ax4.add_feature(cfeature.OCEAN, facecolor='white')

ax4.set_title('d', loc='left', fontsize=12, fontweight='medium')
ax4.set_title('MPI-ESM-LR JJA Max5-MME', loc='right', fontsize=12, fontweight='medium')

ax4.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax4.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax4.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax4.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax4.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax4.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], JJA_max5_trend_mean, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

plt.subplots_adjust(wspace=0.1, hspace=0.01, top=0.75, bottom=0.05)

cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
cb.set_label('Pa/65yrs', fontsize=10, fontweight='medium')
cb.ax.tick_params(labelsize=10)

plt.savefig('Obs-MPI-ESM-LR_1958-2022_JJA_SLP_trend_MME.png', bbox_inches='tight', dpi=300)

plt.close()

/tmp/ipykernel_284438/148840136.py:88: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
